# 💕Day15 Mini Project
Clean a Real Dataset End-to-End

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw = pd.read_csv("/kaggle/input/datasets/shahadmhlalharbi/day15dataset/day15_real_dataset_large.csv")

In [20]:
print(df_raw.head())
print(df_raw.info())
print(df_raw.nunique().sort_values(ascending=False).head(10))

       age    income     city signup_time
0       30   55000.0       NY  2024-01-01
1      NaN   70000.0       SF  2024-01-05
2       45       NaN       LA  not a date
3  unknown  120000.0       NY  2024/02/01
4       28   65000.0  Chicago  2024-01-15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          78 non-null     object 
 1   income       77 non-null     float64
 2   city         96 non-null     object 
 3   signup_time  97 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB
None
signup_time    95
income         62
city           59
age            30
dtype: int64


In [12]:
cleaning_plan = {
    "age": {"type": "float", "missing": "median", "outliers": "cap_99"},
    "income": {"type": "float", "missing": "median", "outliers": "log_cap"},
    "city": {"type": "category", "clean": "lower_strip"},
    "signup_time": {"type": "datetime", "tz": "UTC"}
}

In [14]:
def clean_data_project(df_raw):
    df = df_raw.copy()

    print("Starting cleaning...")

    # Type conversion
    df["age"] = pd.to_numeric(df["age"], errors="coerce")
    df["income"] = pd.to_numeric(df["income"], errors="coerce")
    df["signup_time"] = pd.to_datetime(df["signup_time"], errors="coerce")

    # Missing values
    df["age_missing"] = df["age"].isna().astype(int)
    df["income_missing"] = df["income"].isna().astype(int)

    df["age"] = df["age"].fillna(df["age"].median())
    df["income"] = df["income"].fillna(df["income"].median())

    # Outliers
    cap = df["age"].quantile(0.99)
    df["age"] = df["age"].clip(upper=cap)

    df["income"] = np.log1p(df["income"])
    cap_inc = df["income"].quantile(0.99)
    df["income"] = df["income"].clip(upper=cap_inc)

    # Text cleaning
    df["city"] = (
        df["city"]
        .astype(str)
        .str.strip()
        .str.lower()
    )

    # Datetime
    df["signup_time"] = df["signup_time"].dt.tz_localize("UTC")

    print("Cleaning complete.")
    return df

In [15]:
df_clean = clean_data_project(df_raw)

Starting cleaning...
Cleaning complete.


In [18]:
print(df_clean.info())

print(df_clean[["age","income"]].describe())

print(df_clean["city"].value_counts().head())

print(df_clean["signup_time"].dt.tz)

print(df_clean.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   age             97 non-null     float64            
 1   income          97 non-null     float64            
 2   city            97 non-null     object             
 3   signup_time     42 non-null     datetime64[ns, UTC]
 4   age_missing     97 non-null     int64              
 5   income_missing  97 non-null     int64              
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(1)
memory usage: 4.7+ KB
None
             age     income
count  97.000000  97.000000
mean   34.330309  11.222736
std     9.931294   0.220275
min   -10.000000  10.714440
25%    32.000000  11.097425
50%    35.000000  11.225257
75%    39.000000  11.385103
max    48.040000  11.654398
city
ny         4
sf         4
la         3
austin     3
chicago    2
Name: count, dtype: int64
UTC

In [21]:
df_clean.to_csv("cleaned_dataset.csv", index=False)

# The end🤍